In [40]:
import os
import datetime as dt
import pandas as pd
import shutil

# Get today's date
date_today = dt.datetime.today().strftime('%Y%m%d')

# Define the path to the Incoming_Files directory for today's date
incoming_path = f'Incoming_Files/{date_today}'
success_path = f'Success_Files/{date_today}'
rejected_path = f'Rejected_Files/{date_today}'

# Check if the directory exists
if os.path.isdir(incoming_path):
    # List all files in the directory
    files_in_directory=os.listdir(incoming_path)
    
    # Loop over each file in the directory
    for order in files_in_directory:

        if order.startswith('.') or os.path.isdir(os.path.join(incoming_path, order)):
            continue
        
        header = False
        rejected_count=0
        success_count=0
        
        # Construct the full file path
        path_of_order = os.path.join(incoming_path, order)

        if os.path.isfile(path_of_order):
            df_order = pd.read_csv(path_of_order)
        print(df_order)

        df_product_master=pd.read_csv(f'Product_Master/product_master.csv')
        print(df_product_master)

        product_id_master_list=df_product_master['product_id'].values
        print(product_id_master_list)

        record_reasons = {}

        #Validation1 product_id

        for i in range(len(df_order)):
            value = df_order.iloc[i]['product_id']
            if value not in product_id_master_list:
                pid_reject_reason = f"Invalid product_id {value}"
                if i in record_reasons:
                    record_reasons[i].append(pid_reject_reason)
                else:
                    record_reasons[i] = [pid_reject_reason]

        #Validation2
        combined_df = pd.merge(left = df_order, right = df_product_master, how = 'inner', on = 'product_id')
        print(combined_df)

        # Create a mapping from original dataframe indices to combined dataframe
        original_indices = {}
        for i in range(len(combined_df)):
            # Find matching row in original dataframe
            for j in range(len(df_order)):
                if (df_order.iloc[j]['order_id'] == combined_df.iloc[i]['order_id'] and 
                    df_order.iloc[j]['product_id'] == combined_df.iloc[i]['product_id']):
                    original_indices[i] = j
                    break

        for i in range(len(combined_df)):
            sales_value = combined_df.iloc[i]['sales']
            quantity_value = combined_df.iloc[i]['quantity']
            price_value = combined_df.iloc[i]['price']

            if sales_value != (quantity_value * price_value):
                sales_reject_reason = f"Invalid sales for product_id {combined_df.iloc[i]['product_id']}"
                original_idx = original_indices.get(i)
                if original_idx is not None:
                    if original_idx in record_reasons:
                        record_reasons[original_idx].append(sales_reject_reason)
                    else:
                        record_reasons[original_idx] = [sales_reject_reason]


        #Validation3

        for i in range(len(df_order)):
            order_date_val = df_order.iloc[i]['order_date']
            if order_date_val > date_today:
                date_reject_reason = f"Date is in the future {order_date_val}"
                if i in record_reasons:
                    record_reasons[i].append(date_reject_reason)
                else:
                    record_reasons[i] = [date_reject_reason]

        #Validation4

        for i in range(len(df_order)):
            row = df_order.iloc[i]
            if row.isna().any():
                empty_reject_reason = f"Empty value"
                if i in record_reasons:
                    record_reasons[i].append(empty_reject_reason)
                else:
                    record_reasons[i] = [empty_reject_reason]
            
        #Validation5

        for i in range(len(df_order)):
            city = df_order.iloc[i]['city']
            if city not in ('Mumbai', 'Bangalore'):
                city_reject_reason = f"City {city} not in Mumbai and Bangalore"
                if i in record_reasons:
                    record_reasons[i].append(city_reject_reason)
                else:
                    record_reasons[i] = [city_reject_reason]

        if not record_reasons:
            success_count +=1
            shutil.copy(path_of_order, success_path)
        else:
            
            if not os.path.exists(f'{rejected_path}'):
                os.makedirs(f'{rejected_path}', exist_ok=True)
            shutil.copy(f'{incoming_path}/{order}', f'{rejected_path}/{order}')
            rejected_count += 1

            # Create a copy of the dataframe for rejected records
            rejected_df = df_order.copy()

            # Add rejection reasons column
            rejected_df['rejected_reason'] = ''

            # Fill in the rejection reasons for each record
            for idx, reasons in record_reasons.items():
                rejected_df.at[idx, 'rejected_reason'] = '; '.join(reasons)

            # Filter to only include records with rejection reasons
            rejected_df = rejected_df[rejected_df['rejected_reason'] != '']
            
            # Write to error file
            csv_file_path = f'{rejected_path}/error_{order}'
            rejected_df[['order_id', 'order_date', 'product_id', 'quantity', 'sales', 'city', 'rejected_reason']].to_csv(csv_file_path, index=False)
            
else:
    print(f"Directory not found: {incoming_path}")


   order_id  order_date  product_id  quantity  sales       city
0         1  2023-01-01         100         1  50000  Bangalore
1         2  2023-02-01         200         2   9000     Mumbai
2         3  2023-01-03         200         2   4500     Mumbai
3         4  2023-01-04         800         3  30000     Mysore
   product_id product_name  price     category
0         100       iphone  50000       mobile
1         200        chair   4500    furniture
2         300        table  10000    furniture
3         400           tv  25000  electronics
4         500       fridge  10000  electronics
[100 200 300 400 500]
   order_id  order_date  product_id  quantity  sales       city product_name  \
0         1  2023-01-01         100         1  50000  Bangalore       iphone   
1         2  2023-02-01         200         2   9000     Mumbai        chair   
2         3  2023-01-03         200         2   4500     Mumbai        chair   

   price   category  
0  50000     mobile  
1   4500  f